# Read All Dataset CSV

In [1]:
import os
import csv
import pandas as pd
import numpy as np

In [2]:
dataset_names=[]
X_trains=[]
y_trains=[]
X_tests=[]
for folder_name in os.listdir("./Competition_data"):
    # print(folder_name)
    dataset_names.append(folder_name)
    X_trains.append(pd.read_csv(f"./Competition_data/{folder_name}/X_train.csv",header=0))
    y_trains.append(pd.read_csv(f"./Competition_data/{folder_name}/y_train.csv",header=0))
    X_tests.append(pd.read_csv(f"./Competition_data/{folder_name}/X_test.csv",header=0))

## Data Preprocessing & Feature Engineering

In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer

# 資料前處理與特徵工程
# 我們會進行缺失值處理、標準化與編碼
X_trains_processed = []
X_tests_processed = []

for i in range(len(dataset_names)):
    # 1. 將 X_train 和 X_test 轉換為 DataFrame 格式
    X_train_df = pd.DataFrame(X_trains[i].copy())
    X_test_df = pd.DataFrame(X_tests[i].copy()) 

    # 2. 缺失值處理
    imputer = SimpleImputer(strategy="mean")  #用「均值填補」的方式來處理缺失值，即將資料列中的缺失值填充為該列的平均值。
    X_train_df = imputer.fit_transform(X_train_df)#這一步會在訓練資料集（X_train_df）中計算出每一列的均值並用其填補缺失值。
    X_test_df = imputer.transform(X_test_df)#這一步使用訓練資料集（X_train_df）計算的均值來填補測試資料集（X_test_df）的缺失值。這樣保證測試集的處理與訓練集一致。


    X_train_df = pd.DataFrame(X_train_df)  # 轉回 DataFrame 以便進行 select_dtypes
    X_test_df = pd.DataFrame(X_test_df)    # 轉回 DataFrame 以便進行 select_dtypes

    #目標：將資料中的類別型特徵轉換為數字型資料。原因：許多機器學習模型（如隨機森林、SVM等）只能處理數字資料，因此需要將資料中的類別型特徵（例如文字型的類別）轉換成數字。
    for col in X_train_df.select_dtypes(include=['object']).columns:#所有的 object 類型資料（通常為類別型資料）。這裡假設類別型特徵的資料型態是 object，例如 'A', 'B', 'C' 等。
        encoder = LabelEncoder()
        X_train_df[col] = encoder.fit_transform(X_train_df[col])#對訓練資料（X_train_df）中的每個類別型欄位進行編碼。
        X_test_df[col] = encoder.transform(X_test_df[col])#對測試資料（X_test_df）進行編碼，這樣測試資料的編碼會使用與訓練資料相同的映射。

    # 4. 標準化數值特徵
    scaler = StandardScaler()
    X_train_df = scaler.fit_transform(X_train_df)
    X_test_df = scaler.transform(X_test_df)

    # 將處理過的資料存入 list 中
    X_trains_processed.append(X_train_df)
    X_tests_processed.append(X_test_df)


## train test split & build Model
You can select an appropriate model and perform corresponding hyperparameter tuning.

In [7]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score

In [8]:
for i, (X_train, y_train) in enumerate(zip(X_trains_processed, y_trains)):
    if (X_train.shape[0] != len(y_train)):
        print(f"資料集 {i}: X_train 行數={X_train.shape[0]}, y_train 長度={len(y_train)}")


In [9]:
y_trains = [df['target'] for df in y_trains]

In [10]:
# 設定 XGBoost 模型的參數
params = {
    'objective': 'binary:logistic',  # 二分類任務
    'eval_metric': 'auc',           # 評估指標是 AUC
    'eta': 0.1,                     # 學習率
    'max_depth': 6,                 # 樹的最大深度
    'subsample': 0.8,               # 用於建樹的樣本比例
    'colsample_bytree': 0.8,        # 用於建樹的特徵比例
    'verbosity': 1                  # 是否靜默訓練過程（0: 打印詳細信息，1: 靜默）
}

In [11]:
models = []
for i, (X_train, y_train) in enumerate(zip(X_trains_processed, y_trains)):
    dtrain = xgb.DMatrix(X_train, label=y_train)
    bst = xgb.train(params, dtrain, num_boost_round=100)
    
    models.append(bst)
    print(f"資料集 {dataset_names[i]} 的模型訓練完成")

資料集 Dataset_1 的模型訓練完成
資料集 Dataset_10 的模型訓練完成
資料集 Dataset_11 的模型訓練完成
資料集 Dataset_12 的模型訓練完成
資料集 Dataset_13 的模型訓練完成
資料集 Dataset_14 的模型訓練完成
資料集 Dataset_15 的模型訓練完成
資料集 Dataset_16 的模型訓練完成
資料集 Dataset_17 的模型訓練完成
資料集 Dataset_18 的模型訓練完成
資料集 Dataset_19 的模型訓練完成
資料集 Dataset_2 的模型訓練完成
資料集 Dataset_20 的模型訓練完成
資料集 Dataset_21 的模型訓練完成
資料集 Dataset_22 的模型訓練完成
資料集 Dataset_23 的模型訓練完成
資料集 Dataset_24 的模型訓練完成
資料集 Dataset_25 的模型訓練完成
資料集 Dataset_26 的模型訓練完成
資料集 Dataset_27 的模型訓練完成
資料集 Dataset_28 的模型訓練完成
資料集 Dataset_29 的模型訓練完成
資料集 Dataset_3 的模型訓練完成
資料集 Dataset_30 的模型訓練完成
資料集 Dataset_31 的模型訓練完成
資料集 Dataset_32 的模型訓練完成
資料集 Dataset_33 的模型訓練完成
資料集 Dataset_34 的模型訓練完成
資料集 Dataset_35 的模型訓練完成
資料集 Dataset_36 的模型訓練完成
資料集 Dataset_37 的模型訓練完成
資料集 Dataset_38 的模型訓練完成
資料集 Dataset_39 的模型訓練完成
資料集 Dataset_4 的模型訓練完成
資料集 Dataset_40 的模型訓練完成
資料集 Dataset_41 的模型訓練完成
資料集 Dataset_42 的模型訓練完成
資料集 Dataset_43 的模型訓練完成
資料集 Dataset_44 的模型訓練完成
資料集 Dataset_45 的模型訓練完成
資料集 Dataset_46 的模型訓練完成
資料集 Dataset_47 的模型訓練完成
資料集 Dataset_48 的模型訓練完成
資料集 Dataset_49 

In [12]:
y_preds = []
y_pred_classes = []
for i, bst in enumerate(models):
    dtest = xgb.DMatrix(X_tests_processed[i])
    y_pred = bst.predict(dtest)
    y_pred_class = (y_pred > 0.01).astype(int)
    y_preds.append(y_pred)
    y_pred_classes.append(y_pred_class)

## Inference Model

In [13]:
y_predicts = []
for i in range(len(dataset_names)):
    df = pd.DataFrame(y_pred_classes[i], columns=['y_predict_proba'])
    y_predicts.append(df)
    

## Save result

In [14]:
for idx,dataset_name in enumerate(dataset_names):
    df=y_predicts[idx]
    df.to_csv(f'./Competition_data/{dataset_name}/y_predict.csv', index=False,header=True)

In [15]:
y_predicts

[     y_predict_proba
 0                  1
 1                  1
 2                  1
 3                  1
 4                  1
 ..               ...
 291                1
 292                1
 293                1
 294                1
 295                1
 
 [296 rows x 1 columns],
      y_predict_proba
 0                  1
 1                  1
 2                  1
 3                  1
 4                  1
 ..               ...
 307                1
 308                1
 309                1
 310                1
 311                1
 
 [312 rows x 1 columns],
     y_predict_proba
 0                 1
 1                 1
 2                 1
 3                 1
 4                 1
 5                 1
 6                 1
 7                 1
 8                 1
 9                 1
 10                1
 11                1
 12                1
 13                1
 14                1
 15                1
 16                1
 17                1
 18                